In [1]:
import pandas as pd
import datetime
import numpy as np
from copy import deepcopy
import cProfile
import sys
from collections import defaultdict
from tqdm import tqdm 
import pickle
from bisect import bisect_left
import random
from tqdm import tqdm
import random
import os
import logging


from graph import TransportGraph, ContactionTransportGraph
from forward_search import FCH
from dijkstra import Dijkstra
from timetable import Timetable
from connection_scan_algorithm import ConnectionScanAlgorithm

# Select city

In [16]:
CITY = 'luxembourg'

# Build Transport Graphs

In [17]:
# Download transport connection
transport_connections = pd.read_csv(F'data/{CITY}/network_temporal_day.csv', sep=';')
transport_connections = transport_connections[
    transport_connections['from_stop_I']!=transport_connections['to_stop_I']]

# Download walking connections
walk_connections = pd.read_csv(F'data/{CITY}/network_walk.csv', sep=';')
df_walk_invert = walk_connections.copy()
df_walk_invert = df_walk_invert.rename(columns={'from_stop_I': 'to_stop_I', 'to_stop_I': 'from_stop_I'})
walk_connections = pd.concat((walk_connections, df_walk_invert))

#Additional limit 
walk_connections = walk_connections[walk_connections['d_walk'] < 600]

#Build tg graph
tg = TransportGraph(transport_connections=transport_connections, walk_connections=walk_connections)

In [18]:
NODES = tg.nodes

# Build CH-graph

In [19]:
ch_tg = tg.contraction_hierarchy()

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1367/1367 [00:29<00:00, 47.06it/s]


# Precompute geometrical containers

In [20]:
ch_tg.geometrical_container()

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1367/1367 [00:00<00:00, 14065.97it/s]


# Precompute timetable nodes

### For standard graph

In [21]:
tg_ttn = deepcopy(tg)
tg_ttn_fc_ascending = deepcopy(tg)
tg_ttn_fc_descending = deepcopy(tg)

tg_ttn.optimize_binary_search()

tg_ttn_fc_ascending.fractional_cascading_precomputation('ascending')
tg_ttn_fc_descending.fractional_cascading_precomputation('descending')

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1367/1367 [00:00<00:00, 3329.01it/s]


### For CH-graph

In [22]:
ch_tg_ttn = deepcopy(ch_tg)
ch_tg_ttn_fc_ascending = deepcopy(ch_tg)
ch_tg_ttn_fc_descending = deepcopy(ch_tg)
ch_tg_ttn_fc_hierarchy = deepcopy(ch_tg)

ch_tg_ttn.optimize_binary_search()
ch_tg_ttn_fc_ascending.fractional_cascading_precomputation('ascending')
ch_tg_ttn_fc_descending.fractional_cascading_precomputation('descending')
ch_tg_ttn_fc_hierarchy.fractional_cascading_precomputation('contraction_hierarchy')

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1367/1367 [00:02<00:00, 494.74it/s]


# Prepare data for Connection Scan Algorithm

In [23]:
timetable = Timetable(transport_connections=transport_connections, walk_connections=walk_connections)

# Save Preprocessed Data Structures

In [24]:
# os.mkdir(f'graph/')


# Classic Transport graph
pickle.dump(tg, open(f'graph/{CITY}_tg.pkl', 'wb'), 
            pickle.HIGHEST_PROTOCOL)
pickle.dump(tg_ttn, open(f'graph/{CITY}_tg_ttn.pkl', 'wb'), 
        pickle.HIGHEST_PROTOCOL)
pickle.dump(tg_ttn_fc_ascending, open(f'graph/{CITY}_tg_ttn_fc_ascending.pkl', 'wb'), 
            pickle.HIGHEST_PROTOCOL)
pickle.dump(tg_ttn_fc_descending, open(f'graph/{CITY}_tg_ttn_fc_descending.pkl', 'wb'), 
            pickle.HIGHEST_PROTOCOL)

# CH-graph
pickle.dump(ch_tg, open(f'graph/{CITY}_ch_tg.pkl', 'wb'), 
            pickle.HIGHEST_PROTOCOL)
pickle.dump(ch_tg_ttn, open(f'graph/{CITY}_ch_tg_ttn.pkl', 'wb'), 
            pickle.HIGHEST_PROTOCOL)
pickle.dump(ch_tg_ttn_fc_ascending, open(f'graph/{CITY}_ch_tg_ttn_fc_ascending.pkl', 'wb'), 
            pickle.HIGHEST_PROTOCOL)
pickle.dump(ch_tg_ttn_fc_descending, open(f'graph/{CITY}_ch_tg_ttn_fc_descending.pkl', 'wb'), 
            pickle.HIGHEST_PROTOCOL)
pickle.dump(ch_tg_ttn_fc_hierarchy, open(f'graph/{CITY}_ch_tg_ttn_fc_hierarchy.pkl', 'wb'), 
            pickle.HIGHEST_PROTOCOL)

# CSA precomputation
pickle.dump(timetable, open(f'graph/{CITY}_csa.pkl', 'wb'), 
            pickle.HIGHEST_PROTOCOL)

In [25]:
del tg, tg_ttn, tg_ttn_fc_ascending, tg_ttn_fc_descending, ch_tg, ch_tg_ttn, ch_tg_ttn_fc_ascending, ch_tg_ttn_fc_descending, ch_tg_ttn_fc_hierarchy,timetable 

# Compare solutions

In [30]:
N = 1000
algorithms = [ 'tg', 'tg_ttn', 'tg_ttn_fc_ascending', 'tg_ttn_fc_descending',
         'ch_tg', 'ch_tg_ttn', 'ch_tg_ttn_fc_ascending', 'ch_tg_ttn_fc_descending', 'ch_tg_ttn_fc_hierarchy',
         'csa']
#results = []


test_data = pd.DataFrame({'start_time': [random.randint(transport_connections['dep_time_ut'].min(), 
                                       transport_connections['dep_time_ut'].max()) for i in range(N)],
         'start_node' : [random.sample(NODES, 1)[0] for i in range(N)], 
          'end_node' : [random.sample(NODES, 1)[0] for i in range(N)]
         })
duration = {x: [] for x in algorithms}
explored_nodes = {x: [] for x in algorithms}
trip_duration = {x: [] for x in algorithms}

for algorithm in algorithms:
    graph = pickle.load(open(F'graph/{CITY}_{algorithm}.pkl', 'rb'))
    for index, row in test_data.iterrows():
        if algorithm == 'ch_tg':
            pathfinding = FCH(graph=graph,
                      start_time=row['start_time'],
                      start_node=row['start_node'], 
                      end_node=row['end_node'])
            path = pathfinding.shortest_path(60, optimized_binary_search=False, fractional_cascading=False,
                      fractional_cascading_old=False)
            duration[algorithm].append(path['duration'])
            explored_nodes[algorithm].append(len(pathfinding.candidate_weights))
            if path['path']:
                trip_duration[algorithm].append(path['arrival'] - row['start_time'])
        elif algorithm == 'ch_tg_ttn':
            pathfinding = FCH(graph=graph,
                      start_time=row['start_time'],
                      start_node=row['start_node'], 
                      end_node=row['end_node'])
            path = pathfinding.shortest_path(60, 
                                             optimized_binary_search=True, 
                                             fractional_cascading=False,
                                             fractional_cascading_old=False)
            duration[algorithm].append(path['duration'])
            explored_nodes[algorithm].append(len(pathfinding.candidate_weights))
            if path['path']:
                trip_duration[algorithm].append(path['arrival'] - row['start_time'])
        elif algorithm == 'ch_tg_ttn_fc_ascending': 
            pathfinding = FCH(graph=graph,
                      start_time=row['start_time'],
                      start_node=row['start_node'], 
                      end_node=row['end_node'])
            path = pathfinding.shortest_path(60, 
                                             optimized_binary_search=True, 
                                             fractional_cascading=False,
                                             fractional_cascading_old=True)
            duration[algorithm].append(path['duration'])
            explored_nodes[algorithm].append(len(pathfinding.candidate_weights))
            if path['path']:
                trip_duration[algorithm].append(path['arrival'] - row['start_time'])
        elif algorithm == 'ch_tg_ttn_fc_descending':
            pathfinding = FCH(graph=graph,
                      start_time=row['start_time'],
                      start_node=row['start_node'], 
                      end_node=row['end_node'])
            path = pathfinding.shortest_path(60, 
                                             optimized_binary_search=True, 
                                             fractional_cascading=False,
                                             fractional_cascading_old=True)
            duration[algorithm].append(path['duration'])
            explored_nodes[algorithm].append(len(pathfinding.candidate_weights))
            if path['path']:
                trip_duration[algorithm].append(path['arrival'] - row['start_time'])
            
        elif algorithm == 'ch_tg_ttn_fc_hierarchy':
            pathfinding = FCH(graph=graph,
                      start_time=row['start_time'],
                      start_node=row['start_node'], 
                      end_node=row['end_node'])
            path = pathfinding.shortest_path(60, 
                                             optimized_binary_search=True, 
                                             fractional_cascading=True,
                                             fractional_cascading_old=False)
            duration[algorithm].append(path['duration'])
            explored_nodes[algorithm].append(len(pathfinding.candidate_weights))
            if path['path']:
                trip_duration[algorithm].append(path['arrival'] - row['start_time'])
            
        elif algorithm == 'tg':
            pathfinding = Dijkstra(graph=graph,
                      start_time=row['start_time'],
                      start_node=row['start_node'], 
                      end_node=row['end_node'])
            path = pathfinding.shortest_path(60, optimized_binary_search=False, fractional_cascading=False)
            duration[algorithm].append(path['duration'])
            explored_nodes[algorithm].append(len(pathfinding.candidate_weights))
            if path['path']:
                trip_duration[algorithm].append(path['arrival'] - row['start_time'])
        elif algorithm == 'tg_ttn':
            pathfinding = Dijkstra(graph=graph,
                      start_time=row['start_time'],
                      start_node=row['start_node'], 
                      end_node=row['end_node'])
            path = pathfinding.shortest_path(60, optimized_binary_search=True, fractional_cascading=False)
            duration[algorithm].append(path['duration'])
            explored_nodes[algorithm].append(len(pathfinding.candidate_weights))
            if path['path']:
                trip_duration[algorithm].append(path['arrival'] - row['start_time'])
        elif algorithm =='tg_ttn_fc_ascending':
            pathfinding = Dijkstra(graph=graph,
                      start_time=row['start_time'],
                      start_node=row['start_node'], 
                      end_node=row['end_node'])
            path = pathfinding.shortest_path(60, optimized_binary_search=True, fractional_cascading=True)
            duration[algorithm].append(path['duration'])
            explored_nodes[algorithm].append(len(pathfinding.candidate_weights))
            
            if path['path']:
                trip_duration[algorithm].append(path['arrival'] - row['start_time'])
        elif algorithm == 'tg_ttn_fc_descending':
            pathfinding = Dijkstra(graph=graph,
                      start_time=row['start_time'],
                      start_node=row['start_node'], 
                      end_node=row['end_node'])
            path = pathfinding.shortest_path(60, optimized_binary_search=True, fractional_cascading=True)
            duration[algorithm].append(path['duration'])
            explored_nodes[algorithm].append(len(pathfinding.candidate_weights))
            
            if path['path']:
                trip_duration[algorithm].append(path['arrival'] - row['start_time'])
        elif algorithm == 'csa':
            pathfinding = ConnectionScanAlgorithm(graph=graph,
                  start_time=row['start_time'],
                  start_node=row['start_node'], 
                  end_node=row['end_node'])
            path = pathfinding.shortest_path()
            duration[algorithm].append(path['duration'])
            
            if path['path']:
                trip_duration[algorithm].append(path['arrival'] - row['start_time'])


/var/folders/kt/g2cs84fs4j3c1psy_vtfg4h40000gn/T/ipykernel_62497/3755359733.py:10: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  'start_node' : [random.sample(NODES, 1)[0] for i in range(N)],
/var/folders/kt/g2cs84fs4j3c1psy_vtfg4h40000gn/T/ipykernel_62497/3755359733.py:11: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  'end_node' : [random.sample(NODES, 1)[0] for i in range(N)]


# Results

In [31]:
for algorithm in algorithms:
    print(algorithm, np.mean(duration[algorithm]), np.std(duration[algorithm]))

tg 8.879 5.667835477499325
tg_ttn 7.255 4.772627682943642
tg_ttn_fc_ascending 7.36 4.888394419438759
tg_ttn_fc_descending 7.691 10.79877395818618
ch_tg 4.201 2.1933077759402577
ch_tg_ttn 2.559 1.3588668073067354
ch_tg_ttn_fc_ascending 4.093 2.1176286265537683
ch_tg_ttn_fc_descending 3.879 2.0204848428038256
ch_tg_ttn_fc_hierarchy 3.433 1.854591868848777
csa 5.455 4.548403566087776
